**Mount Drive & Start Spark**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BTS_Flight_Ingestion") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .getOrCreate()

print("Spark Version:", spark.version)

Spark Version: 4.0.2


**Configure Paths**

In [ ]:
import os

DRIVE_BASE   = "/content/drive/MyDrive/Machine Learning and Big Data"
PARQUET_PATH = "/content/drive/MyDrive/Machine Learning and Big Data/parquet_data"
TABLEAU_PATH = "/content/drive/MyDrive/Machine Learning and Big Data/tableau_exports"
CHECKPOINT   = "/content/drive/MyDrive/Machine Learning and Big Data/checkpoints"
LOGS_PATH    = "/content/drive/MyDrive/Machine Learning and Big Data/logs"

for path in [PARQUET_PATH, TABLEAU_PATH, CHECKPOINT, LOGS_PATH]:
    os.makedirs(path, exist_ok=True)

spark.sparkContext.setCheckpointDir(CHECKPOINT)

print("Paths configured")
print("CSV Source :", DRIVE_BASE)
print("Parquet Out:", PARQUET_PATH)

Paths configured
CSV Source : /content/drive/MyDrive/Machine Learning and Big Data
Parquet Out: /content/drive/MyDrive/Machine Learning and Big Data/parquet_data


**List Available Files**

In [ ]:
csv_files = sorted([f for f in os.listdir(DRIVE_BASE) if f.endswith('.csv')])

print(f"Found {len(csv_files)} CSV file(s):\n")
for f in csv_files:
    size_mb = os.path.getsize(os.path.join(DRIVE_BASE, f)) / (1024**2)
    print(f"  {f}  →  {size_mb:.1f} MB")

Found 6 CSV file(s):

  On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_1.csv  →  235.4 MB
  On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_2.csv  →  223.5 MB
  On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_3.csv  →  255.3 MB
  On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_4.csv  →  251.0 MB
  On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_5.csv  →  263.6 MB
  On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_6.csv  →  264.0 MB


**Data Ingestion**

In [ ]:
from pyspark.sql import functions as F

# Map month number to filename
month_files = {
    1: "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_1.csv",
    2: "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_2.csv",
    3: "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_3.csv",
    4: "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_4.csv",
    5: "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_5.csv",
    6: "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2024_6.csv",
}

dfs = []
for month, filename in month_files.items():
    path = os.path.join(DRIVE_BASE, filename)
    if os.path.exists(path):
        df_month = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .option("nullValue", "") \
            .option("nanValue", "NA") \
            .csv(path)
        dfs.append(df_month)
        print(f"  Month {month}: {df_month.count():,} rows  |  {len(df_month.columns)} columns")
    else:
        print(f"  Month {month}: FILE NOT FOUND — {filename}")

print(f"\nTotal files loaded: {len(dfs)}")

  Month 1: 547,271 rows  |  110 columns
  Month 2: 519,221 rows  |  110 columns
  Month 3: 591,767 rows  |  110 columns
  Month 4: 582,185 rows  |  110 columns
  Month 5: 609,743 rows  |  110 columns
  Month 6: 611,132 rows  |  110 columns

Total files loaded: 6


**Combine All Months**

In [ ]:
from functools import reduce
from pyspark.sql import DataFrame

df = reduce(DataFrame.unionByName, dfs)

print("Combined Dataset")
print("  Total Rows   :", df.count())
print("  Total Columns:", len(df.columns))
print("  Partitions   :", df.rdd.getNumPartitions())

Combined Dataset
  Total Rows   : 3461319
  Total Columns: 110
  Partitions   : 14


**Data Schema Overview**

In [ ]:
# Print full schema
print("=" * 60)
print("DATASET SCHEMA")
print("=" * 60)
df.printSchema()

# Collect schema as structured data for export
schema_rows = []
for field in df.schema.fields:
    schema_rows.append({
        "column_name": field.name,
        "data_type":   str(field.dataType),
        "nullable":    field.nullable
    })

print(f"\nTotal columns: {len(schema_rows)}")
print(f"\n{'Column':<45} {'Type':<20} {'Nullable'}")
print("-" * 75)
for row in schema_rows:
    print(f"  {row['column_name']:<43} {row['data_type']:<20} {row['nullable']}")

DATASET SCHEMA
root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: date (nullable = true)
 |-- Reporting_Airline: string (nullable = true)
 |-- DOT_ID_Reporting_Airline: integer (nullable = true)
 |-- IATA_CODE_Reporting_Airline: string (nullable = true)
 |-- Tail_Number: string (nullable = true)
 |-- Flight_Number_Reporting_Airline: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- OriginAirportSeqID: integer (nullable = true)
 |-- OriginCityMarketID: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- Des

**Data Validation**

In [ ]:
raw_count = df.count()

# Rule 1: Valid Year and Month
df = df.filter(
    F.col("Year").isNotNull() &
    F.col("Month").isNotNull() &
    F.col("Year").between(1987, 2025) &
    F.col("Month").between(1, 12)
)

# Rule 2: Origin and Destination must exist
df = df.filter(
    F.col("Origin").isNotNull() &
    F.col("Dest").isNotNull()
)

# Rule 3: Distance must be positive
df = df.filter(
    F.col("Distance").isNull() | (F.col("Distance") > 0)
)

valid_count    = df.count()
rejected_count = raw_count - valid_count

print("Validation Summary")
print(f"  Raw rows     : {raw_count:,}")
print(f"  Valid rows   : {valid_count:,}")
print(f"  Rejected rows: {rejected_count:,}  ({100*rejected_count/raw_count:.2f}%)")

Validation Summary
  Raw rows     : 3,461,319
  Valid rows   : 3,461,319
  Rejected rows: 0  (0.00%)


**Repartition & Persist**

In [ ]:
from pyspark import StorageLevel

# Partition by Month + Airline — aligns with downstream query patterns
df = df.repartition(12, "Month", "Reporting_Airline")

print("Repartitioned by Month + Reporting_Airline")
print("  Partition count:", df.rdd.getNumPartitions())

# Persist before multiple downstream actions (EDA exports + Parquet write)
df.persist(StorageLevel.MEMORY_AND_DISK)
_ = df.count()
print("DataFrame persisted to MEMORY_AND_DISK")

Repartitioned by Month + Reporting_Airline
  Partition count: 12
DataFrame persisted to MEMORY_AND_DISK


**Save Parquet**

In [ ]:
df.write \
  .mode("overwrite") \
  .partitionBy("Year", "Month") \
  .parquet(PARQUET_PATH)

print("Parquet saved")
print("  Location   :", PARQUET_PATH)
print("  Partitioned: Year / Month")
print("  Compression: Snappy")

verify_df = spark.read.parquet(PARQUET_PATH)
print("\nVerification:")
print("  Rows   :", verify_df.count())
print("  Columns:", len(verify_df.columns))

df.unpersist()
print("\nDataFrame unpersisted — executor memory freed")

Parquet saved
  Location   : /content/drive/MyDrive/Machine Learning and Big Data/parquet_data
  Partitioned: Year / Month
  Compression: Snappy

Verification:
  Rows   : 3461319
  Columns: 110

DataFrame unpersisted — executor memory freed


**Reload & Tune Partitions**

In [ ]:
df = spark.read.parquet(PARQUET_PATH)

row_count          = df.count()
current_partitions = df.rdd.getNumPartitions()
optimal_partitions = max(4, row_count // 100_000)

df = df.repartition(optimal_partitions)

print("Shuffle & Partition Tuning")
print(f"  Rows              : {row_count:,}")
print(f"  Previous partitions: {current_partitions}")
print(f"  Optimal partitions : {optimal_partitions}")
print(f"  Target             : ~100k rows per partition")

# Re-cache for all Tableau CSV exports
df.cache()
_ = df.count()
print("DataFrame re-cached for Tableau exports")

Shuffle & Partition Tuning
  Rows              : 3,461,319
  Previous partitions: 3
  Optimal partitions : 34
  Target             : ~100k rows per partition
DataFrame re-cached for Tableau exports


**Tableau CSV: Dashboard 1 — Data Quality & Pipeline Monitoring**

In [ ]:
# Dataset Overview

from pyspark.sql import Row

dataset_overview_data = [
    Row(metric="Total Raw Rows", value=raw_count),
    Row(metric="Valid Rows", value=valid_count),
    Row(metric="Rejected Rows", value=rejected_count),
    Row(metric="Total Columns", value=len(df.columns)),
    Row(metric="Total Partitions", value=df.rdd.getNumPartitions())
]

dataset_overview_df = spark.createDataFrame(dataset_overview_data)

dataset_overview_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(os.path.join(TABLEAU_PATH, "d1_dataset_overview"))

In [ ]:
# Validation Funnel

validation_funnel_data = [
    Row(stage="Raw Ingestion", rows=raw_count),
    Row(stage="After Validation Rules", rows=valid_count)
]

validation_funnel_df = spark.createDataFrame(validation_funnel_data)

validation_funnel_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(os.path.join(TABLEAU_PATH, "d1_validation_funnel"))

In [ ]:
# Missing Values Summary

from pyspark.sql.functions import col, sum as spark_sum, when

missing_exprs = [
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns
]

missing_counts_df = df.agg(*missing_exprs)

# Convert wide → long format
missing_long_df = missing_counts_df.selectExpr(
    "stack({0}, {1}) as (column, null_count)".format(
        len(df.columns),
        ",".join([f"'{c}', `{c}`" for c in df.columns])
    )
)

total_rows = df.count()

missing_final_df = missing_long_df.withColumn(
    "pct_missing",
    (col("null_count") / total_rows) * 100
)

missing_final_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(os.path.join(TABLEAU_PATH, "d1_missing_values"))

In [ ]:
# Monthly Pipeline Monitoring

monthly_pipeline_df = df.groupBy("Month").agg(
    F.count("*").alias("total_flights"),
    F.sum(F.when(F.col("Cancelled") == 1, 1).otherwise(0)).alias("cancelled_flights"),
    F.sum(F.when(F.col("Diverted") == 1, 1).otherwise(0)).alias("diverted_flights"),
    F.avg("ArrDelay").alias("avg_arrival_delay")
).orderBy("Month")

monthly_pipeline_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(os.path.join(TABLEAU_PATH, "d1_monthly_pipeline"))